In [1]:
import os
import json
import time
import psutil
from contextlib import contextmanager
import pickle

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import lightgbm as lgb

def show_memory_usage(name = "unknown"):
    vm = psutil.virtual_memory()
    print(f"[MEMUSE] memory usage (in {name}): {vm.used/1024/1024:.2f}MB ({vm.percent}%)")

@contextmanager
def timer(name: str):
    show_memory_usage(f"before {name}")
    s = time.time()
    yield
    elapsed = time.time() - s
    print(f"[{name}] {elapsed:.3f}sec")
    show_memory_usage(f"after {name}")
    
class Config:
    is_train = True
    is_debug = True
    is_rerun = os.getenv('KAGGLE_IS_COMPETITION_RERUN')
    src_path = "/kaggle/input/cmisleep-2ndplace-kmat"
    train_path = '/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet'
    train_target_path = '/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv'
    test_path = '/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet'
    sample_submission_path = '/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv'
    
    lgbm_path = "/kaggle/input/cmisleep-2ndplace-kmat/trained_model/lgbm_models"
    # weights
    lgbm_paths_2nd = [[os.path.join(lgbm_path, "lgb_acm_0_th30"),
                       os.path.join(lgbm_path, "lgb_acm_0_th50"),
                       os.path.join(lgbm_path, "lgb_acm_0_th70"),
                      ],
                      [os.path.join(lgbm_path, "lgb_acm_1_th30"),
                       os.path.join(lgbm_path, "lgb_acm_1_th50"),
                       os.path.join(lgbm_path, "lgb_acm_1_th70"),
                      ],
                     ]
    lgbm_sub_paths_2nd = [[os.path.join(lgbm_path, "lgb_simple_0")],
                          [os.path.join(lgbm_path, "lgb_simple_1")]]
    
    lgbm_path_third = os.path.join(lgbm_path, "lgb_third")
    
    preprocess_dir = 'data_processed/'
    pred_dirs = ['preds_0/', 'preds_1/']
    steps_per_sec = 0.2
    step_for_a_day = 60 * steps_per_sec * 60 * 24
    step_for_30min = 60 * steps_per_sec * 30
    step_for_15min = 60 * steps_per_sec * 15
    step_for_1min = 60 * steps_per_sec
    data_for_debug = 100
    
    pred_overlap = 0.5
    pred_batch = 48

cfg = Config()
os.makedirs(cfg.preprocess_dir, exist_ok=True)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
%load_ext cython

In [ ]:
%%cython
import numpy as np
cimport numpy as cnp
cimport cython

def cumsum_morethan_zero(cnp.ndarray[cnp.float64_t, ndim=1] x):
    cdef int i, n
    n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] y = np.zeros(n)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] y_rev = np.zeros(n)
    y[0] = x[0]
    for i in range(1, n):
        if x[i] == 0:
            y[i] = 0
        else:
            y[i] = y[i-1] + x[i]
    y_rev[-1] = y[-1]
    for i in range(n-2, -1, -1):
        if y_rev[i+1] > y[i]:
            if x[i] == 0:
                y_rev[i] = 0
            else:
                y_rev[i] = y_rev[i+1]
        else:
            y_rev[i] = y[i]
    return y_rev

def easy_convolve(cnp.ndarray[cnp.float64_t, ndim=1] x, int filter_size):
    """
    padding same, kernel is ones
    """
    cdef int i, j, n, p, m
    m = filter_size - 1
    p = m // 2
    n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] x_p = np.zeros(n+2*p)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] y = np.zeros(n)
    x_p[p:n+p] = x

    for j in range(filter_size):
        y[0] += x_p[j]

    for i in range(1, n):# filter_size, n+p+p-filter_size+1):
        y[i] = x_p[i+m] + y[i-1] - x_p[i-1]
    return y

def minimum(cnp.ndarray[cnp.float64_t, ndim=1] x, cnp.float64_t maxval):
    cdef int i, n
    n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] y = np.zeros(n)
    for i in range(n):
        y[i] = min(x[i], maxval)
    return y

def easy_closing(cnp.ndarray[cnp.float64_t, ndim=1] x, int filter_size):
    """
    closing = dilation -> erosion
    padding same, kernel is ones, x is 0 or 1
    """
    x = easy_convolve(x, filter_size)
    x = minimum(x, 1)
    x = 1 - x
    x = easy_convolve(x, filter_size)
    x = minimum(x, 1)
    x = 1 - x
    return x

def easy_closing_q(cnp.ndarray[cnp.float64_t, ndim=1] x, int filter_size):
    """
    closing = dilation -> erosion
    padding same, kernel is ones, x is 0 or 1
    """
    cdef int i, j, n, p, m
    m = filter_size - 1
    p = m // 2
    n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] x_p = np.zeros(n+2*p)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] y_p = np.zeros(n+2*p)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] y = np.zeros(n)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] z = np.zeros(n)
    
    x_p[p:n+p] = x
    for j in range(filter_size):
        y[0] += x_p[j]
    for i in range(1, n):# filter_size, n+p+p-filter_size+1):
        y[i] = x_p[i+m] + y[i-1] - x_p[i-1]
    for i in range(n):
        y[i] = 1 - min(y[i], 1)
    
    y_p[p:n+p] = y
    for j in range(filter_size):
        z[0] += y_p[j]
    for i in range(1, n):# filter_size, n+p+p-filter_size+1):
        z[i] = y_p[i+m] + z[i-1] - y_p[i-1]
    for i in range(n):
        z[i] = 1 - min(z[i], 1)
    
    return z


def detect_peak(cnp.ndarray[cnp.float64_t, ndim=1] x, int k):
    cdef int n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] max_array = np.zeros(n, dtype=np.float64)
    cdef cnp.ndarray[cnp.int32_t, ndim=1] max_indices = np.zeros(n, dtype=np.int32)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] result = np.zeros(n, dtype=np.float64)
    cdef int i, j, start, end, max_index
    
    # calculate max values in each window
    for i in range(n):
        start = max(0, i - k)
        end = min(n, i + k + 1)
        max_index = start
        for j in range(start, end):
            if x[j] > x[max_index]:
                max_index = j
        max_array[i] = x[max_index]
        max_indices[i] = max_index
    
    # set peak values to 1
    for i in range(n):
        if x[i] == max_array[max_indices[i]]:
            result[i] = 1.0
    
    return max_array

def detect_peak_r(cnp.ndarray[cnp.float64_t, ndim=1] x, int k):
    cdef int n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] max_array = np.zeros(n, dtype=np.float64)
    cdef cnp.ndarray[cnp.int32_t, ndim=1] max_indices = np.zeros(n, dtype=np.int32)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] result = np.zeros(n, dtype=np.float64)
    cdef int i, j, start, end, max_index
    
    # calculate max values in first window
    max_index = 0
    for i in range(k):
        if x[i] > x[max_index]:
            max_index = i
    max_array[k-1] = x[max_index]
    max_indices[k-1] = max_index
    
    # calculate max values in each window
    for i in range(k, n):
        start = i - k
        end = i
        if max_index == start - 1:
            max_index = start
            for j in range(start, end):
                if x[j] > x[max_index]:
                    max_index = j
        else:
            if x[i] > x[max_index]:
                max_index = i
        max_array[i] = x[max_index]
        max_indices[i] = max_index
    
    # set peak values to 1
    for i in range(n):
        if x[i] == max_array[max_indices[i]]:
            result[i] = 1.0
    
    return max_array

@cython.boundscheck(False)
@cython.wraparound(False)
def detect_peak_kmat(cnp.ndarray[cnp.float64_t, ndim=1] x, int k):
    cdef int n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] max_array = np.zeros(n, dtype=np.float64)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] result_val = np.zeros(n, dtype=np.float64)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] result = np.zeros(n, dtype=np.float64)
    cdef int i, j, start, end, max_index, half_k

    half_k = k // 2
    for i in range(half_k, n-half_k):
        result[i] = 1
        result_val[i] = x[i]
        for j in range(1, half_k+1):
            if x[i] < x[i-j]:
                result[i] = 0
                result_val[i] = 0
                break
            if x[i] < x[i+j]:
                result[i] = 0
                result_val[i] = 0
                break
    return result, result_val


In [ ]:
def timestamp_to_step_single_id(df_train_id, df_target_id=None):
    """
    Convert timestamp to step
    timestepは2018-08-14T22:26:00-0400といった形で与えられる
    
    """
    offset_date = df_train_id['timestamp'].iloc[0:1].str.split('T', expand=True)[0]
    offset_time = df_train_id['timestamp'].iloc[0:1].str.split('T', expand=True)[1].str.split('-', expand=True)[0]
    offset = pd.to_datetime(offset_date + ' ' + offset_time)
    offset_step = offset.dt.hour * 60 * 12 + offset.dt.minute * 12 + offset.dt.second / 5
    df_train_id["daily_step"] = (df_train_id['step'] + offset_step.values[0]) % cfg.step_for_a_day
    if df_target_id is not None:
        df_target_id["daily_step"] = (df_target_id['step'] + offset_step.values[0]) % cfg.step_for_a_day
    # print((df['step'].iloc[-1] + offset_step) % cfg.step_for_a_day)
    return df_train_id, df_target_id

def sample_train_target_by_id(df_train, df_target, id_no):
    return df_train[df_train['series_id']==id_no], df_target[df_target['series_id']==id_no]


def find_sensor_error(df_train_id, column='anglez'):
    
    def padding(x, left, right, value=0):
        return np.pad(x, (left, right), 'constant', constant_values=(value, value))


    x = df_train_id['daily_step'].values
    y = df_train_id[column].values
    mask = np.ones_like(y)
    pad_left = int(x[0])
    pad_right = int(cfg.step_for_a_day) -1 - int(x[-1]) % int(cfg.step_for_a_day)
    
    x = padding(x, pad_left, pad_right, value=0)
    y = padding(y, pad_left, pad_right, value=0)
    y_dif = padding(np.abs(y[1:] - y[:-1]), 1, 0, value=0)
    mask = padding(mask, pad_left, pad_right, value=0)


    # reshape to (num_day, step_for_a_day)
    x = x.reshape(-1, int(cfg.step_for_a_day))
    y = y.reshape(-1, int(cfg.step_for_a_day))
    y_dif = y_dif.reshape(-1, int(cfg.step_for_a_day))
    mask = mask.reshape(-1, int(cfg.step_for_a_day))
    day_counter = np.cumsum(mask, axis=0) # (num_day, step_for_a_day)

    # メモリ注意。
    delta_matrix = y[np.newaxis, :, :] - y[:, np.newaxis, :] # (num_day, num_day, step_for_a_day)
    mask_matrix = mask[np.newaxis, :, :] * mask[:, np.newaxis, :] # (num_day, num_day, step_for_a_day)
    delta_matrix = (delta_matrix==0) * mask_matrix # where delta is zero except for the padding area
    
    delta_matrix = np.sum(delta_matrix, axis=0) - 1 # (num_day, step_for_a_day). 同じデータ同士の差は0になるので-1
    mask_matrix = np.sum(mask_matrix, axis=0) # (num_day, step_for_a_day)
    nan_counter = np.cumsum(delta_matrix > 0, axis=0) # (num_day, step_for_a_day) 初回のnan風なものは案外ラベルがついている？
    nan_exist_other_day = np.any(delta_matrix > 0, axis=0, keepdims=True) # (1, step_for_a_day)
    nan_exist_other_day = np.tile(nan_exist_other_day, (delta_matrix.shape[0], 1)) # (num_day, step_for_a_day)

    maybe_not_nan = delta_matrix == 0
    valid = mask * maybe_not_nan
    y_valid = y_dif * maybe_not_nan # (num_day, step_for_a_day) ★一時的にy_difを使う。
    y_sum = np.sum(y_valid, axis=0, keepdims=True) # (1, step_for_a_day)
    y_mean = y_sum / (np.sum(valid, axis=0, keepdims=True)+1e-7) # (1, step_for_a_day)
    y_dev = (y_valid - y_mean) * valid # (num_day, step_for_a_day)
    y_std = np.sqrt(np.sum(y_dev**2, axis=0, keepdims=True) / (np.sum(valid, axis=0, keepdims=True)+1e-7)) # (1, step_for_a_day)
    # mean, stdをtileして、yと同じshapeにする
    y_mean = np.tile(y_mean, (y.shape[0], 1))
    y_std = np.tile(y_std, (y.shape[0], 1)) # (num_day, step_for_a_day)


    # flatten
    delta_matrix = delta_matrix.reshape(-1)
    mask_matrix = mask_matrix.reshape(-1)
    day_counter = day_counter.reshape(-1)
    nan_counter = nan_counter.reshape(-1)
    nan_exist_other_day = nan_exist_other_day.reshape(-1)
    y_mean = y_mean.reshape(-1)
    y_std = y_std.reshape(-1)

    # left, rightのpadding部分を除く
    start = pad_left
    end = -pad_right if pad_right > 0 else len(delta_matrix)
    delta_matrix = delta_matrix[start:end]
    mask_matrix = mask_matrix[start:end]
    day_counter = day_counter[start:end]
    nan_counter = nan_counter[start:end]
    nan_exist_other_day = nan_exist_other_day[start:end]
    y_mean = y_mean[start:end]
    y_std = y_std[start:end]

    df_train_id[column+"_numrepeat"] = delta_matrix
    df_train_id[column+"_daycount"] = mask_matrix
    df_train_id[column+"_daycounter"] = day_counter
    df_train_id[column+"_nancounter"] = nan_counter
    df_train_id[column+"_nanexist"] = nan_exist_other_day
    df_train_id[column+"_daymean"] = y_mean
    df_train_id[column+"_daystd"] = y_std

    # smoothing 5step closing    
    df_train_id[column+"_simpleerror"] = easy_closing_q((delta_matrix > 0).astype(np.float64),  5) # 5step
    df_train_id[column+"_simpleerror_span"] = cumsum_morethan_zero(df_train_id[column+"_simpleerror"].values)
    df_train_id[column+"_simpleerror_v2"] = df_train_id[column+"_simpleerror"].astype(int) + (df_train_id[column+"_simpleerror_span"] > cfg.step_for_30min).astype(int) + (df_train_id[column+"_simpleerror_span"] > (cfg.step_for_30min*2)).astype(int) # 

    return df_train_id

def make_binary_target(df_train_id, df_target_id, id_no):
    df_train_id['target'] = -1
    
    current_state = -1
    current_step = 0
    # state would continues no less than 1 or 2hour 
    continue_length = int(cfg.step_for_30min*4)

    for total_step, event_type, night_no in zip(df_target_id['step'].values, df_target_id['event'].values, df_target_id['night'].values):
        tmp = current_state
        data_exist = not np.isnan(total_step)
        if event_type == 'onset' and data_exist:
            if current_state == 1 or current_state == -1:
                df_train_id.loc[(df_train_id['step'] >= current_step) & (df_train_id['step'] < total_step), 'target'] = 1
            if current_state == -2: # nan before onset. 
                df_train_id.loc[(df_train_id['step'] >= np.maximum(total_step - continue_length, 0)) & (df_train_id['step'] < total_step), 'target'] = 1
            current_state = 0
            current_step = total_step
        elif event_type == 'wakeup' and data_exist:
            if current_state == 0 or current_state == -1:
                df_train_id.loc[(df_train_id['step'] >= current_step) & (df_train_id['step'] < total_step), 'target'] = 0
            if current_state == -2: # nan before wakeup. 
                df_train_id.loc[(df_train_id['step'] >= np.maximum(total_step - continue_length, 0)) & (df_train_id['step'] < total_step), 'target'] = 0
            current_state = 1
            current_step = total_step
        elif not data_exist: # nan
            if current_state == 1: # nan after wakeup
                df_train_id.loc[(df_train_id['step'] >= current_step) & (df_train_id['step'] < current_step+continue_length), 'target'] = 1
            elif current_state == 0: # nan after onset
                df_train_id.loc[(df_train_id['step'] >= current_step) & (df_train_id['step'] < current_step+continue_length), 'target'] = 0

            current_state = -2
        # print(tmp, "->", event_type, "->", current_state, data_exist, total_step)

    if current_state == 1: # nan after wakeup
        df_train_id.loc[(df_train_id['step'] >= current_step) & (df_train_id['step'] < current_step+continue_length), 'target'] = 1
        end_step_for_train = current_step+continue_length
    elif current_state == 0: # nan after onset
        df_train_id.loc[(df_train_id['step'] >= current_step) & (df_train_id['step'] < current_step+continue_length), 'target'] = 0
        end_step_for_train = current_step+continue_length
    
    elif current_state == -2:
        end_step_for_train = (night_no - 1) * cfg.step_for_a_day

    
    return df_train_id, end_step_for_train


def prerprocess_inputs_for_mlmodel(df_train_id):
    df_train_id["daily_step"] = (df_train_id["daily_step"] / cfg.step_for_a_day).astype(np.float32)
    df_train_id["anglez"] = (df_train_id["anglez"] / 90).astype(np.float32)
    df_train_id["anglez_simpleerror"] = df_train_id["anglez_simpleerror"].astype(np.float32)
    # anglez_shimpleerror_spanはlog1pをとっておく
    df_train_id["anglez_simpleerror_span"] = np.clip(np.log1p(df_train_id["anglez_simpleerror_span"]) / 10, 0, 1).astype(np.float32)
    df_train_id["anglez_nanexist"] = df_train_id["anglez_nanexist"].astype(np.float32)
    df_train_id["anglez_daystd"] = np.clip(df_train_id["anglez_daystd"] / 90, 0, 2).astype(np.float32)
    df_train_id["anglez_daymean"] = np.clip(df_train_id["anglez_daymean"] / 90, 0, 2).astype(np.float32)
    df_train_id["anglez_daycounter"] = (df_train_id["anglez_daycounter"] / df_train_id["anglez_daycounter"].max()).astype(np.float32)
    df_train_id["anglez_nancounter"] = (df_train_id["anglez_nancounter"]>2).astype(np.float32)
    max_daystd = np.max(df_train_id["anglez_daystd"].values)
    max_daymean = np.max(df_train_id["anglez_daymean"].values)
    if max_daystd > 10:
        print("max_daystd > 10", max_daystd)
    if max_daymean > 10:
        print("max_daymean > 10", max_daymean)
        

    df_train_id["enmo"] = np.clip(np.log1p(df_train_id["enmo"]), 0, 5).astype(np.float32)

    if "target" in df_train_id.columns:
        df_train_id["target"] = df_train_id["target"].astype(np.float32)
    return df_train_id

def make_dataset(df_train_id, id_no, df_target_id=None):
    if df_target_id is None:
        make_train_data = False
    else:
        make_train_data = True
    file_path = os.path.join(cfg.preprocess_dir, f"id_{id_no}_feature.npy")
    file_step_path = os.path.join(cfg.preprocess_dir, f"id_{id_no}_step.npy")
    file_path_meta = os.path.join(cfg.preprocess_dir, f"id_{id_no}_meta.json")
    train_columns = ['daily_step', 'anglez', 'anglez_simpleerror', 'anglez_simpleerror_span', 
    'anglez_nanexist', 'anglez_daystd', 'anglez_daymean', "anglez_daycounter", "anglez_nancounter", 
    'enmo']
    # if make_train_data:
    #     train_columns += ['target']
    
    # with timer(f"select_dataset {id_no}"):
    #     df_train_id, df_target_id = sample_train_target_by_id(df_train, df_target, id_no)
    # with timer(f"timestamp_to_step {id_no}"):
    # df_train_id = timestamp_to_step(df_train_id)
    # df_target_id = timestamp_to_step(df_target_id)
    df_train_id, df_target_id = timestamp_to_step_single_id(df_train_id, df_target_id)
    # with timer(f"find_sensor_error {id_no}"):
    df_train_id = find_sensor_error(df_train_id, column='anglez')
    if make_train_data:
        # with timer(f"make_binary_target {id_no}"):
        df_train_id, end_step_for_train = make_binary_target(df_train_id, df_target_id, id_no)
        meta_data = {"end_step_for_train": end_step_for_train}

    df_train_id = prerprocess_inputs_for_mlmodel(df_train_id)

    # numpyに変換して保存する
    np.save(file_path, df_train_id[train_columns].values)
    np.save(file_step_path, df_train_id["step"].values)
    if make_train_data:
        with open(file_path_meta, 'w') as f:
            json.dump(meta_data, f, indent=4)



In [ ]:

import os
import gc

import pandas as pd
import numpy as np


def timestamp_to_step(df):
    """
    Convert timestamp to step
    timestepは2018-08-14T22:26:00-0400といった形で与えられる
    """
    df['date'] = df['timestamp'].str.split('T', expand=True)[0]
    df['time'] = df['timestamp'].str.split('T', expand=True)[1].str.split('-', expand=True)[0]
    df['timestamp_dt'] = pd.to_datetime(df['date']+' '+df['time'])
    df['daily_step'] = df['timestamp_dt'].dt.hour * 60 * 12 + df['timestamp_dt'].dt.minute * 12 + df['timestamp_dt'].dt.second / 5
    return df


def data_preparation(load_path, load_path_target=None, save_dir = "cacheTMP/"):
    os.makedirs(save_dir, exist_ok=True)

    # series_idsの代わり目のindexを取得し、それをもとにparquetのファイルを分割して読み込み、処理をおこなう
    # df = pd.read_parquet(load_path) -> これはOOM
    series_ids = pd.read_parquet(load_path, columns=["series_id"])["series_id"].values
    unique_ids, unique_indices = np.unique(series_ids, return_index=True)
    unique_indices = np.sort(unique_indices)
    sorted_ids = series_ids[unique_indices]
    start_indices = unique_indices
    end_indices = np.append(unique_indices[1:], len(series_ids))
    del series_ids
    
    if load_path_target is None:
        df_target = None
    else:
        df_target = pd.read_csv(cfg.train_target_path)

    load_step = 5
    starts = np.arange(0, len(sorted_ids), load_step)
    ends = np.append(starts[1:], len(sorted_ids))
    for start, end in zip(starts, ends):
        chunk_series_ids = sorted_ids[start:end]
        chunk_start_ind = start_indices[start:end] - start_indices[start]
        chunk_end_ind = end_indices[start:end] - start_indices[start]        
        print(f"start: {start}, end: {end}")
        filters = [("series_id", "in", chunk_series_ids)]
        chunk = pd.read_parquet(load_path, filters=filters)
        for start_index, end_index, series_id in zip(chunk_start_ind, chunk_end_ind, chunk_series_ids):
            chunk_sub = chunk.iloc[start_index:end_index].copy()
            make_dataset(chunk_sub, series_id, df_target) # npyで保存
            # chunk_sub.to_parquet(os.path.join(save_dir, f"{series_id}.parquet"), index=False)
        if cfg.is_train and cfg.is_debug:
            if end>=cfg.data_for_debug:
                break
    del chunk
    del chunk_sub
    del end_indices, start_indices, sorted_ids, unique_indices, unique_ids
    gc.collect()
    


In [ ]:
import sys
import glob

import tensorflow as tf
from tensorflow.keras import backend as K

SRC_PATH = os.path.join(cfg.src_path, 'src')
sys.path.insert(1, SRC_PATH)

from train_1dcnn import SleepAwake, set_seeds
    
def build_model(weight_file, length, model_type="normal"):
    K.clear_session()
    
    
    model_params = {"input_shape": (length, 10),
                    "output_shape": (length, 1),  
                    "weight_file": weight_file,
                    "is_train_model": False,
                    "model_type":model_type}
    sa_model = SleepAwake(**model_params)
    return sa_model

    

def infer_predict_twin(load_dir, save_dir_0, save_dir_1, overlap=0.9):
    show_memory_usage("before model load")
    set_seeds(111)

    weight_files_0 = [os.path.join(cfg.src_path, f"trained_model/exp00_run_00_SplitStem5foldSEED111controledStride_fold{i}/final_weights.h5") for i in range(5)]
    weight_files_0 += [os.path.join(cfg.src_path, f"trained_model/exp00_run_00_SplitStem5foldSEED42controledStride_fold{i}/final_weights.h5") for i in range(5)]

    weight_files_1 = [os.path.join(cfg.src_path, f"trained_model/exp00_run_01_SplitStem5foldSEED111normal_fold{i}/final_weights.h5") for i in range(5)]
    weight_files_1 += [os.path.join(cfg.src_path, f"trained_model/exp00_run_01_SplitStem5foldSEED42normal_fold{i}/final_weights.h5") for i in range(5)]

    
    sa_models_0 = [build_model(weight_file=w, length=2880 * 5, model_type="contstride") for w in weight_files_0]
    num_ensemble_0 = len(sa_models_0)

    sa_models_1 = [build_model(weight_file=w, length=1024 * 14, model_type="normal") for w in weight_files_1]
    num_ensemble_1 = len(sa_models_1)

    npy_files = sorted(glob.glob(os.path.join(load_dir, "*feature.npy")))
    npy_files_step = sorted(glob.glob(os.path.join(load_dir, "*step.npy")))
    show_memory_usage("before inference")
    if not os.path.exists(save_dir_0):
        os.makedirs(save_dir_0)
    if not os.path.exists(save_dir_1):
        os.makedirs(save_dir_1)
    for i, (f, fs) in enumerate(zip(npy_files, npy_files_step)):
        show_memory_usage(f"inference {i}")

        stt = time.time()
        features = np.load(f)# [:end_step]
        steps = np.load(fs)# [:end_step]
        inputs = features[:,:] # features[:,:-1] if is_train else features[:,:]
        daily_step = features[:, 0]
        pred_nan = features[:, 2]
        nan_span = features[:, 3]
        nan_counter = features[:, -4]
        
        # 2/4 models        
        # preds, preds_switch = sa_models[0].overlap_predict(inputs, overlap=overlap) # oom
        preds, preds_switch = sa_models_0[0].overlap_predict_no_oom(inputs, overlap=overlap,  max_batch = cfg.pred_batch)
        for m in sa_models_0[1:]:
            preds_ensemble, preds_switch_ensemble = m.overlap_predict_no_oom(inputs, overlap=overlap, max_batch=cfg.pred_batch)
            preds += preds_ensemble
            preds_switch += preds_switch_ensemble
        preds /= num_ensemble_0
        preds_switch /= num_ensemble_0
        
        # print("pred time", time.time()-stt)
        stt = time.time()

        # parquetで保存。csvよりも軽いはやい
        save_path = save_dir_0 + os.path.basename(f).replace(".npy", ".parquet")
        # df = pd.DataFrame({"step": steps, "daily_step": daily_step, "pred_awake": preds.flatten(), "pred_switch": preds_switch.flatten(), "pred_nan": pred_nan})
        STRIDE = 1
        df = pd.DataFrame({"step": steps[:len(preds_switch)*STRIDE][::STRIDE],
                                "daily_step": daily_step[:len(preds_switch)*STRIDE][::STRIDE], 
                                "pred_awake": preds.flatten().astype(np.float32), 
                                "pred_switch10p": preds_switch[:,0].astype(np.float32), 
                                "pred_switch10": preds_switch[:,1].astype(np.float32), 
                                "pred_switch8": preds_switch[:,2].astype(np.float32),
                                "pred_switch6": preds_switch[:,3].astype(np.float32),
                                "pred_switch4": preds_switch[:,4].astype(np.float32),
                                "pred_switch2": preds_switch[:,5].astype(np.float32),
                                "pred_nan": pred_nan[:len(preds_switch)*STRIDE][::STRIDE].astype(np.float32),
                                "pred_nan_span": nan_span[:len(preds_switch)*STRIDE][::STRIDE].astype(np.float32),
                                "pred_nan_counter": nan_counter[:len(preds_switch)*STRIDE][::STRIDE].astype(np.float32),
                                })
        df.to_parquet(save_path, index=False)
        
        del df, preds, preds_switch, preds_ensemble, preds_switch_ensemble
        
        # 2/4 models
        preds, preds_switch = sa_models_1[0].overlap_predict_no_oom(inputs, overlap=overlap,  max_batch = 32)
        for m in sa_models_1[1:]:
            preds_ensemble, preds_switch_ensemble = m.overlap_predict_no_oom(inputs, overlap=overlap, max_batch=32)
            preds += preds_ensemble
            preds_switch += preds_switch_ensemble
        preds /= num_ensemble_1
        preds_switch /= num_ensemble_1
        
        # print("pred time", time.time()-stt)
        stt = time.time()

        save_path = save_dir_1 + os.path.basename(f).replace(".npy", ".parquet")
        STRIDE = 1
        df = pd.DataFrame({"step": steps[:len(preds_switch)*STRIDE][::STRIDE],
                                "daily_step": daily_step[:len(preds_switch)*STRIDE][::STRIDE], 
                                "pred_awake": preds.flatten().astype(np.float32), 
                                "pred_switch10p": preds_switch[:,0].astype(np.float32), 
                                "pred_switch10": preds_switch[:,1].astype(np.float32), 
                                "pred_switch8": preds_switch[:,2].astype(np.float32),
                                "pred_switch6": preds_switch[:,3].astype(np.float32),
                                "pred_switch4": preds_switch[:,4].astype(np.float32),
                                "pred_switch2": preds_switch[:,5].astype(np.float32),
                                "pred_nan": pred_nan[:len(preds_switch)*STRIDE][::STRIDE].astype(np.float32),
                                "pred_nan_span": nan_span[:len(preds_switch)*STRIDE][::STRIDE].astype(np.float32),
                                "pred_nan_counter": nan_counter[:len(preds_switch)*STRIDE][::STRIDE].astype(np.float32),
                                })
        df.to_parquet(save_path, index=False)
        
        del df, preds, preds_switch, preds_ensemble, preds_switch_ensemble
        del features, steps, inputs, daily_step, pred_nan
        if i%2==0:
            gc.collect()
    del sa_models_0
    del sa_models_1

            

## POSTPROCESS

In [ ]:
import glob
import os
import time

import tensorflow as tf
import scipy
import numpy as np
import pandas as pd



def detect_peak(array, valid_mask, kernel_size=5, threshold = 0.05):
    peak_mask, peak_array = detect_peak_kmat(array.astype(np.float64), int(kernel_size))
    pred_mask = peak_mask * valid_mask
    peak_array = peak_array * valid_mask
    peak_indices = np.where(peak_array>0)[0]
    peak_val = peak_array[peak_indices]
    mask_over_threshold = peak_val > threshold
    peak_indices = peak_indices[mask_over_threshold]
    peak_val = peak_val[mask_over_threshold]
    peak_results = {"peak_array": peak_array, "peak_indices": peak_indices, "peak_val": peak_val}
    return peak_results

def gaussian_smooth(array, sigma):#
    if sigma<=0:
        return array
    array = scipy.ndimage.gaussian_filter1d(array, sigma, mode='reflect')
    return array


def smooth_awake(pred_awake, gauss_factor):
    """
    gaussian smooth
    """
    
    return smooth_pred_awake

def find_event(event_indices, pred_awake, before_length, after_length, awake_threshold=0.5, sleep_threshold=0.5, awake_sleep_diff_threshold=0, lengthlist_for_2ndstage=None, pred_nan=None, pred_scores=None):
    """
    event stepの前後の状態の平均値をとり、それが閾値を超えているかどうかで状態変化を判定する
    TODO 平均じゃないほうがいいかも？
    """
    before_states = []
    after_states = []
    event_types = []
    for event_index in event_indices:
        before_state = pred_awake[max(0, event_index-before_length):event_index].mean()
        after_state = pred_awake[event_index+1:min(event_index+after_length+1, len(pred_awake))].mean()
        before_states.append(before_state)
        after_states.append(after_state)
        if before_state > after_state + awake_sleep_diff_threshold:
            if before_state > awake_threshold and after_state < sleep_threshold:
                event_type = "onset"
            else:
                event_type = "nan"
        elif after_state > before_state + awake_sleep_diff_threshold:
            if before_state < sleep_threshold and after_state > awake_threshold:
                event_type = "wakeup"
            else:
                event_type = "nan"
        else:
            event_type = "nan"
        event_types.append(event_type)
    event_results = {"event_indices": event_indices, "before_states": before_states, "after_states": after_states, "event_types": event_types}


    if lengthlist_for_2ndstage is not None:
        for length in lengthlist_for_2ndstage:
            event_results[f"before_states_feat_{length}"] = [pred_awake[max(0, event_index-length):event_index].mean() for event_index in event_indices]
            event_results[f"after_states_feat_{length}"] = [pred_awake[event_index+1:min(event_index+length+1, len(pred_awake))].mean() for event_index in event_indices]
            if pred_nan is not None:
                event_results[f"before_nan_feat_{length}"] = [pred_nan[max(0, event_index-length):event_index].mean() for event_index in event_indices]
                event_results[f"after_nan_feat_{length}"] = [pred_nan[event_index+1:min(event_index+length+1, len(pred_awake))].mean() for event_index in event_indices]
            if pred_scores is not None:
                for i, col in enumerate(['pred_switch', "pred_switch10p", "pred_switch10", "pred_switch8", "pred_switch6", "pred_switch4", "pred_switch2"]):
                    # event_results[f"before_{col}_{length//2}"] = [pred_scores[max(0, event_index-length//2):event_index,i].mean() for event_index in event_indices]
                    # event_results[f"after_{col}_{length//2}"] = [pred_scores[event_index+1:min(event_index+length//2+1, len(pred_awake)),i].mean() for event_index in event_indices]
                    event_results[f"befaf_{col}_{length//2}"] = [pred_scores[max(0, event_index-length//2):event_index,i].mean()-pred_scores[event_index+1:min(event_index+length//2+1, len(pred_awake)),i].mean() for event_index in event_indices]
 
    return event_results

def add_night_group(out_df_single):
    # 10000step (abou 2 pm) -> night += 1
    out_df_single["offset_step"] = out_df_single['step'] + out_df_single['daily_step'].iloc[0] * cfg.step_for_a_day - out_df_single['step'].iloc[0]
    out_df_single["night"] = 1 + (out_df_single["offset_step"] + cfg.step_for_a_day - 10000) // cfg.step_for_a_day
    return out_df_single

def add_night_features(out_df_single, columns=["pred_nan_counter", "pred_nan_span", "pred_nan"]):
    """
    groupby night features for 2nd stage
    """
    out_df_single = add_night_group(out_df_single)
    for col in columns:
        out_df_single[f"{col}_mean"] = out_df_single.groupby("night")[col].transform("mean")
        out_df_single[f"{col}_min"] = out_df_single.groupby("night")[col].transform("min")
        out_df_single[f"{col}_max"] = out_df_single.groupby("night")[col].transform("max")
    # aggregationで実施してからconcatする場合
    # df_agg = out_df_single.groupby("night")[columns].agg(["mean", "min", "max"])
    # df_agg.columns = [f"{col}_{agg}" for col, agg in df_agg.columns]
    # df_agg = df_agg.reset_index()
    # out_df_single = pd.merge(out_df_single, df_agg, on="night", how="left")
    
    out_df_single = out_df_single.drop(columns=["offset_step", "night"])
    return out_df_single


def avoid_6_vals(out_df_single, pred_switch, window_size=20):
    """
    6の倍数のところは評価指標上僅かに損をする。(GTが1分単位であり、評価指標が12または6の倍数であるため)
    予測結果のstepが6の倍数の場合は、そこからどちらかにずらすと得をする。
    """
    # out_df_single['step'] = out_df_single['step'] + ((out_df_single['step']%6) == 0).astype(int)
    out_df_single.reset_index(drop=True, inplace=True)
    out_df_single["is_6_multiple"] = (out_df_single['step']%6) == 0
    shifts = np.zeros(len(out_df_single))
    for i in range(len(out_df_single)-1):
        if out_df_single['is_6_multiple'].iloc[i] == True:
            index = out_df_single['peak_indices'].iloc[i]
            before = pred_switch[max(0, index-window_size):index].mean()
            after = pred_switch[index+1:min(index+window_size+1, len(pred_switch))].mean()
            if before > after:
                # out_df_single['step'].iloc[i] -= 1
                # out_df_single.at[i, 'step']. -= 1
                shifts[i] = -1
            else:
                # out_df_single['step'].iloc[i] += 1
                # out_df_single.at[i, 'step']. += 1
                shifts[i] = 1
    out_df_single['step'] = out_df_single['step'] + shifts.astype(int)

    # drop column
    out_df_single = out_df_single.drop(columns=["is_6_multiple"])
    return out_df_single


def predict_averaging(predictions, weights):
    
    pred = (predictions * np.array(weights).reshape(1,-1)).sum(axis=1)# / np.sum(weights)

    return pred


def make_dataset_for_second_model(df, df_target=None, peak_kernel_size=30, peak_threshold=0.05, awake_threshold=0.5, awake_sleep_diff_threshold=0, sleep_threshold=0.5, event_before_length=30, event_after_length=30, prior_conf_dev=0.1, averaging_weight=[0,1,0.5,0,0,0]):

    lengthlist_for_2ndstage = [12, 24, 60, 120, 240, 360, 720] # used to make features of state
    night_agg_cols = ["pred_nan_counter", "pred_nan_span", "pred_nan", "pred_switch2", "pred_switch", "pred_awake"]
    
    df["pred_switch"] = predict_averaging(df[["pred_switch10p", "pred_switch10", "pred_switch8", "pred_switch6", "pred_switch4", "pred_switch2"]].values, weights=averaging_weight)
    df["pred_switch"] = gaussian_smooth(df["pred_switch"].values, sigma=4)
    df = add_night_features(df, columns=night_agg_cols)
    nonnan_mask = 1 - df["pred_nan"].values
    peak_results = detect_peak(df['pred_switch'].values, nonnan_mask, kernel_size=int(1+2*peak_kernel_size*cfg.step_for_1min), threshold=peak_threshold)
    event_results = find_event(peak_results["peak_indices"], df['pred_awake'].values, int(event_before_length*cfg.step_for_1min), int(event_after_length*cfg.step_for_1min), awake_threshold=awake_threshold, sleep_threshold=sleep_threshold, 
                                awake_sleep_diff_threshold=awake_sleep_diff_threshold, 
                                lengthlist_for_2ndstage=lengthlist_for_2ndstage,
                                pred_nan=df['pred_nan'].values,
                              pred_scores=df[['pred_switch', "pred_switch10p", "pred_switch10", "pred_switch8", "pred_switch6", "pred_switch4", "pred_switch2"]].values,)
    out_df_single = pd.DataFrame({"step": df['step'].values[peak_results["peak_indices"]], 
            "peak_indices": peak_results["peak_indices"], 
            "daily_step": df['daily_step'].values[peak_results["peak_indices"]], 
            "event": event_results['event_types'], 
            "score": peak_results['peak_val'],
            "score_10p": df['pred_switch10p'].values[peak_results["peak_indices"]],
            "score_10": df['pred_switch10'].values[peak_results["peak_indices"]],
            "score_8": df['pred_switch8'].values[peak_results["peak_indices"]],
            "score_6": df['pred_switch6'].values[peak_results["peak_indices"]],
            "score_4": df['pred_switch4'].values[peak_results["peak_indices"]],
            "score_2": df['pred_switch2'].values[peak_results["peak_indices"]],
            "pred_nan": df['pred_nan'].values[peak_results["peak_indices"]],
            "pred_nan_counter": df['pred_nan_counter'].values[peak_results["peak_indices"]],
            "pred_nan_span": df['pred_nan_span'].values[peak_results["peak_indices"]],
            "pred_awake": df['pred_awake'].values[peak_results["peak_indices"]],
            "before_states": event_results['before_states'],
            "after_states": event_results['after_states'],
            })
    for length in lengthlist_for_2ndstage:
        out_df_single[f"before_states_feat_{length}"] = event_results[f"before_states_feat_{length}"]
        out_df_single[f"after_states_feat_{length}"] = event_results[f"after_states_feat_{length}"]
        out_df_single[f"before_nan_feat_{length}"] = event_results[f"before_nan_feat_{length}"]
        out_df_single[f"after_nan_feat_{length}"] = event_results[f"after_nan_feat_{length}"]
        for i, col in enumerate(['pred_switch', "pred_switch10p", "pred_switch10", "pred_switch8", "pred_switch6", "pred_switch4", "pred_switch2"]):
            out_df_single[f"befaf_{col}_{length//2}"] = event_results[f"befaf_{col}_{length//2}"]
    for col in night_agg_cols:
        out_df_single[f"{col}_mean"] = df[f"{col}_mean"].values[peak_results["peak_indices"]]
        out_df_single[f"{col}_min"] = df[f"{col}_min"].values[peak_results["peak_indices"]]
        out_df_single[f"{col}_max"] = df[f"{col}_max"].values[peak_results["peak_indices"]]
                    
    out_df_single = out_df_single[out_df_single['event']!="nan"]
    if len(out_df_single) > 0:
        out_df_single = add_night_group(out_df_single)

    if len(out_df_single) > 0:
        out_df_single = avoid_6_vals(out_df_single, df['pred_switch'].values)
    
    # add_target(not used in inference)
    def diff_step_to_AP(diff_step):
        return np.sum(np.array([diff_step < threshold for threshold in [12, 36, 60, 90, 120, 150, 180, 240, 300, 360]]), axis=0)
    if df_target is not None:
        if len(df_target) >0:
            diff_step_matrix = df_target['step'].values.reshape(-1,1) - out_df_single['step'].values.reshape(1,-1)
            diff_step_matrix = np.abs(diff_step_matrix)
            min_diff_step = np.min(diff_step_matrix, axis=0)
            out_df_single['min_diff_step'] = min_diff_step
            out_df_single['best_ap'] = diff_step_to_AP(min_diff_step)/ 10.
        else:
            out_df_single['min_diff_step'] = np.nan
            out_df_single['best_ap'] = 0

    return out_df_single


def run_postprocess_and_prepare_2nd(pred_dir, pp_params, save_path="df_second_model.feather"):
    pred_files = glob.glob(os.path.join(pred_dir, "*.parquet"))
    out_df = []
    for file in pred_files:
        id_no = os.path.basename(file).split("_")[1]
        df_pred_id = pd.read_parquet(file)
        out_df_single = make_dataset_for_second_model(df_pred_id, **pp_params)
        
        out_df_single['series_id'] = id_no
        out_df.append(out_df_single)
    # out_df = pd.concat(out_df).reset_index(drop=True).reset_index().rename(columns={"index": "row_id"})
    
    out_df = pd.concat(out_df).reset_index(drop=True)
    float32_cols = [col for col in out_df.columns if out_df[col].dtype=="float64"]
    out_df[float32_cols] = out_df[float32_cols].astype(np.float32)
    out_df.to_feather(save_path)
    return out_df


def evaluate_ap(out_df, train_events):
    # out_df = pd.concat(out_df).reset_index(drop=True).reset_index().rename(columns={"index": "row_id"})
    
    series_ids = out_df['series_id'].unique()
    solution =  train_events.loc[train_events['series_id'].isin(series_ids)].copy()
    solution = solution[~np.isnan(solution['step'])]
    total_score = event_detection_ap(solution, out_df.copy(), tolerances)
    return total_score

In [ ]:
def make_features_2ndstage(df, cfg, phase="train"):


    drop_cols = []
    added_cols = []
    score_keys = ['score', 'score_10p', 'score_10', 'score_8', 'score_6', 'score_4', 'score_2']
    
    df["event"] = (df["event"]=="wakeup").astype(int)
    # daily_step for night
    df["daily_step_sleep"] = (df["daily_step"] + 0.5) % 1

    # change of state between before and after
    lengthlist = [12, 24, 60, 120, 240, 360, 720]
    for length in lengthlist:
        df[f"state_diff_{length}"] = df[f"before_states_feat_{length}"] - df[f"after_states_feat_{length}"]
        df[f"nan_diff_{length}"] = df[f"before_nan_feat_{length}"] - df[f"after_nan_feat_{length}"]

    # largest score
    for key in score_keys:
        df[f"max_{key}_sne"] = df.groupby(["series_id", "night", "event"])[key].transform("max")
        df[f"max_{key}_sne_diff"] = df[f"max_{key}_sne"] - df[key]
        df[f"max_{key}_sne_is_peak"] = (df[f"max_{key}_sne_diff"] == 0).astype(int)
        df[f"sum_{key}_sne"] = df.groupby(["series_id", "night", "event"])[key].transform("sum")
        df[f"mean_{key}_sne"] = df.groupby(["series_id", "night", "event"])[key].transform("mean")
        drop_cols.append(f"max_{key}_sne_is_peak")
        added_cols += [f"max_{key}_sne", f"max_{key}_sne_diff", f"max_{key}_sne_is_peak"]

    # largest score
    for key in score_keys:
        df[f"max_{key}_sn"] = df.groupby(["series_id", "night"])[key].transform("max")
        df[f"max_{key}_sn_diff"] = df[f"max_{key}_sn"] - df[key]
        added_cols += [f"max_{key}_sn", f"max_{key}_sn_diff"]
    
    for key in score_keys:
        df_peak = df[df[f"max_{key}_sne_is_peak"] == 1]
        
        df_peak = df_peak.groupby(["series_id", "event"])[f"max_{key}_sne"].agg(["mean", "std"]).reset_index()
        df_peak.columns = ["series_id", "event", f"max_{key}_sne_mean", f"max_{key}_sne_std"]
        df = df.merge(df_peak, on=["series_id", "event"], how="left")
        # normalize
        df[f"{key}_relative_to_peak"] = (df[key] - df[f"max_{key}_sne_mean"]) / df[f"max_{key}_sne_std"]

        added_cols += [f"max_{key}_sne_mean", f"max_{key}_sne_std", f"{key}_relative_to_peak"]

    # daily_step at peak
    for key in score_keys:
        df_peak = df[df[f"max_{key}_sne_is_peak"] == 1].copy()
        df_peak = df_peak.rename(columns={"daily_step": f"peak_daily_step_{key}", "daily_step_sleep": f"peak_daily_step_sleep_{key}"})
        df_peak[f"peak_daily_step_{key}_mean"] = df_peak.groupby(["series_id", "event"])[f"peak_daily_step_{key}"].transform("mean")
        df_peak[f"peak_daily_step_sleep_{key}_mean"] = df_peak.groupby(["series_id", "event"])[f"peak_daily_step_sleep_{key}"].transform("mean") # scoreが高いものだけに限定してもいいのかもな…。

        df = df.merge(df_peak[["series_id", "night", "event", f"peak_daily_step_{key}", f"peak_daily_step_sleep_{key}", f"peak_daily_step_{key}_mean", f"peak_daily_step_sleep_{key}_mean"]], on=["series_id", "night", "event"], how="left")
        df[f"step_dist_from_peak_{key}"] = df["daily_step"] - df[f"peak_daily_step_{key}"]
        df[f"step_dist_from_peak_sleep_{key}"] = df["daily_step_sleep"] - df[f"peak_daily_step_sleep_{key}"]
        df[f"step_dist_from_peak_{key}_mean"] = df["daily_step"] - df[f"peak_daily_step_{key}_mean"]
        df[f"step_dist_from_peak_sleep_{key}_mean"] = df["daily_step_sleep"] - df[f"peak_daily_step_sleep_{key}_mean"]

        added_cols += [f"peak_daily_step_{key}", f"peak_daily_step_sleep_{key}", f"peak_daily_step_{key}_mean", f"peak_daily_step_sleep_{key}_mean", f"step_dist_from_peak_{key}", f"step_dist_from_peak_sleep_{key}", f"step_dist_from_peak_{key}_mean", f"step_dist_from_peak_sleep_{key}_mean"]
        

    # other event at same night
    df_flip = df.copy()
    flip_columns = [f"max_{key}_sne" for key in score_keys]+ [f"sum_{key}_sne" for key in score_keys] + [f"peak_daily_step_{key}" for key in score_keys] + [f"peak_daily_step_sleep_{key}" for key in score_keys]
    df_flip["event"] = 1 - df_flip["event"] #.apply(lambda x: "onset" if x == "wakeup" else "wakeup")
    df_flip = df_flip.groupby(["series_id", "event", "night"])[flip_columns].max().reset_index()
    df_flip.columns = ["series_id", "event", "night"] + [f"{c}_flip" for c in flip_columns]
    df = df.merge(df_flip, on=["series_id", "night", "event"], how="left")

    for key in score_keys:
        df[f"peak_daily_step_{key}_sleep_duration_01"] = df["daily_step"] + 0.5 - df[f"peak_daily_step_sleep_{key}_flip"]
        df[f"peak_daily_step_{key}_sleep_duration_10"] = df[f"peak_daily_step_{key}_flip"] + 0.5 - df["daily_step_sleep"]
        added_cols += [f"peak_daily_step_{key}_sleep_duration_01", f"peak_daily_step_{key}_sleep_duration_10"]

    for key in ['score', 'score_10p', 'score_10', 'score_8', 'score_6', 'score_4', 'score_2']:
        df[f"rank_{key}_sne"] = df.groupby(["series_id", "night", "event"])[key].transform("rank")

    # sort by scoreでsort -> accumulated score
    df = df.sort_values(["series_id", "night", "event", "score"], ascending=False).reset_index(drop=True)
    for key in ['score', 'score_10p', 'score_10', 'score_8', 'score_6', 'score_4', 'score_2']:
        df[f"cumsum_{key}_sne"] = df.groupby(["series_id", "night", "event"])[key].transform("cumsum")
        # 累積の積ももとめる。
        # df[f"tmp"] = 1 - df[key]
        # df[f"cumprod_{key}_sne"] = df.groupby(["series_id", "night", "event"])["tmp"].transform("cumprod")
        # drop_cols.append("tmp")

        # df[f"cumsum_{key}_sne_rate"] = df[key] / df[f"cumsum_{key}_sne"]

    return df

In [ ]:
class LabelEncoders:
    def __init__(self):
        self.encoders = {}
    
    def fit(self, df):
        for c in df:
            if df[c].dtype.name == "object":
                enc = self.encoders.get(c, LabelEncoder())
                enc.fit(df[c])
                self.encoders[c] = enc
                
    def transform(self, df):
        for c in df:
            if c in self.encoders:
                df[c] = self.encoders[c].transform(df[c])
        return df
    
    def fit_one(self, s):
        enc = self.encoders.get(s.name, LabelEncoder())
        enc.fit(s)
        self.encoders[s.name] = enc
        
    def transform_one(self, s):
        if s.name in self.encoders:
            return self.encoders[s.name].transform(s)
        else:
            return s

    def fit_transform(self, df):
        self.fit(df)
        return self.transform(df)
                
    def fit_transform_one(self, s):
        self.fit_one(s)
        return self.transform_one(s)

class LGBMSerializer:
    def __init__(self,
                 booster: lgb.CVBooster,
                 encoders: LabelEncoders,
                 ):
        self.booster = booster
        self.encoders = encoders
        
    
    def to_file(self, filename: str):
        model = {
            "boosters": [b.model_to_string() for b in self.booster.boosters],
            "best_iteration": self.booster.best_iteration,
            
        }
        
        with open(f"{filename}_model.json", "w") as f:
            json.dump(model, f)
            
        with open(f"{filename}_encoder.bin", "wb") as f:
            pickle.dump(self.encoders, f)
            
    @classmethod
    def from_file(cls, filename: str) -> "TrainedModel":
        
        with open(f"{filename}_model.json", "r") as f:
            model = json.load(f)

        cvbooster = lgb.CVBooster()
        cvbooster.boosters = [lgb.Booster(model_str=b) for b in model["boosters"]]
        cvbooster.best_iteration = model["best_iteration"]
        for b in cvbooster.boosters:
            b.best_iteration = cvbooster.best_iteration
            
        with open(f"{filename}_encoder.bin", "rb") as f:
            encoders = pickle.load(f)
            
        return cls(cvbooster, encoders)
            
    
def run_2nd_stage(df, pretrained_dir):
    # make features and predict 
    df = make_features_2ndstage(df, cfg, phase="test")
    serializer = LGBMSerializer.from_file(pretrained_dir)
    cvbooster = serializer.booster
    encoder = serializer.encoders
    feature_cols = cvbooster.feature_name()[0]
    predicted = np.array(cvbooster.predict(encoder.transform(df[feature_cols]))).mean(axis=0)
    
    # make submission
    sub_df = df[["series_id", "step", "event", "score"]].copy()
    sub_df = sub_df.reset_index(drop=True).reset_index().rename(columns={"index": "row_id"})
    # wakeupとonsetに書き換える
    sub_df["event"] = sub_df["event"].apply(lambda x: "onset" if x == 0 else "wakeup")
    sub_df["score"] = ((sub_df["score"]**0.8) + predicted) / 2
    return sub_df

def run_2nd_stage_twin(df, pretrained_dir, pretrained_dir_sub, save_dir, exist_third_stage=False):
    # make features and predict 
    df = make_features_2ndstage(df, cfg, phase="test")
    # sort by score
    df = df.sort_values(["series_id", "night", "event", "score"], ascending=False).reset_index(drop=True)
    
    serializer = LGBMSerializer.from_file(pretrained_dir)
    cvbooster = serializer.booster
    encoder = serializer.encoders
    feature_cols = cvbooster.feature_name()[0]
    predicted = np.array(cvbooster.predict(encoder.transform(df[feature_cols]))).mean(axis=0)

    serializer = LGBMSerializer.from_file(pretrained_dir_sub)
    cvbooster = serializer.booster
    encoder = serializer.encoders
    feature_cols = cvbooster.feature_name()[0]
    predicted_sub = np.array(cvbooster.predict(encoder.transform(df[feature_cols]))).mean(axis=0)
    
    
    # make submission
    if exist_third_stage:
        sub_df = df.copy()
    else:
        sub_df = df[["series_id", "step", "event", "night", "score"]].copy()
    
    
    def postprocess_2nd_stage(df, oof_acm, oof_simple, weights=[1, 0.4, 1]):
        print("need to be sorted by score in advance")
        df["origin_score"] = df["score"].copy()
        df["event"] = df["event"].apply(lambda x: "onset" if x == 0 else "wakeup")
        df["pred_best_ap_until"] = oof_acm
        df["pred_best_ap_until"] = df.groupby(["series_id", "night", "event"])["pred_best_ap_until"].transform(lambda x: np.maximum.accumulate(x)) # avoid minus diff
        df["pred_score_acm"] = df.groupby(["series_id", "night", "event"])["pred_best_ap_until"].diff().fillna(df["pred_best_ap_until"])
        df["pred_score_simple"] = oof_simple
        df["score"] = (df["pred_score_acm"]*weights[0] + df["pred_score_simple"]*weights[1]  + df["origin_score"]*weights[2]) / sum(weights)
        return df
    
    sub_df = postprocess_2nd_stage(sub_df, predicted, predicted_sub, weights=[1, 0.4, 1])
    sub_df.to_feather(save_dir)
    
    if not exist_third_stage:
        # sub_df = sub_df.drop(columns=["pred_score_simple", "pred_score_acm", "pred_best_ap_until", "origin_score"])
        # else:
        sub_df = sub_df.drop(columns=["pred_score_simple", "pred_score_acm", "pred_best_ap_until", "origin_score", "night"])
        sub_df = sub_df.reset_index(drop=True).reset_index().rename(columns={"index": "row_id"})
    return sub_df

def run_2nd_stage_twin_ensemble(df, pretrained_dirs, pretrained_dirs_sub, save_path, exist_third_stage=False):
    # make features and predict 
    df = make_features_2ndstage(df, cfg, phase="test")
    # sort by score
    df = df.sort_values(["series_id", "night", "event", "score"], ascending=False).reset_index(drop=True)
    predicted = []
    for pretrained_dir in pretrained_dirs:
        serializer = LGBMSerializer.from_file(pretrained_dir)
        cvbooster = serializer.booster
        encoder = serializer.encoders
        feature_cols = cvbooster.feature_name()[0]
        predicted.append(np.array(cvbooster.predict(encoder.transform(df[feature_cols]))).mean(axis=0))
    predicted = np.array(predicted).mean(axis=0)
    
    predicted_sub = []
    for pretrained_dir_sub in pretrained_dirs_sub:
        serializer = LGBMSerializer.from_file(pretrained_dir_sub)
        cvbooster = serializer.booster
        encoder = serializer.encoders
        feature_cols = cvbooster.feature_name()[0]
        predicted_sub.append(np.array(cvbooster.predict(encoder.transform(df[feature_cols]))).mean(axis=0))
    predicted_sub = np.array(predicted_sub).mean(axis=0)

    
    # make submission
    if exist_third_stage:
        sub_df = df.copy()
    else:
        sub_df = df[["series_id", "step", "event", "night", "score"]].copy()
    
    
    def postprocess_2nd_stage(df, oof_acm, oof_simple, weights=[1, 0.4, 1]):
        print("need to be sorted by score in advance")
        df["origin_score"] = df["score"].copy()
        df["event"] = df["event"].apply(lambda x: "onset" if x == 0 else "wakeup")
        df["pred_best_ap_until"] = oof_acm
        df["pred_best_ap_until"] = df.groupby(["series_id", "night", "event"])["pred_best_ap_until"].transform(lambda x: np.maximum.accumulate(x)) # avoid minus diff
        df["pred_score_acm"] = df.groupby(["series_id", "night", "event"])["pred_best_ap_until"].diff().fillna(df["pred_best_ap_until"])
        df["pred_score_simple"] = oof_simple
        df["score"] = (df["pred_score_acm"]*weights[0] + df["pred_score_simple"]*weights[1]  + df["origin_score"]*weights[2]) / sum(weights)
        return df
    
    sub_df = postprocess_2nd_stage(sub_df, predicted, predicted_sub, weights=[1, 0.4, 1])
    sub_df.to_feather(save_path)
    
    if not exist_third_stage:
        # sub_df = sub_df.drop(columns=["pred_score_simple", "pred_score_acm", "pred_best_ap_until", "origin_score"])
        # else:
        sub_df = sub_df.drop(columns=["pred_score_simple", "pred_score_acm", "pred_best_ap_until", "origin_score", "night"])
        sub_df = sub_df.reset_index(drop=True).reset_index().rename(columns={"index": "row_id"})
    return sub_df


# ensemble after 2nd stage
def weighted_fusion_ensemble(df_0, df_1, distance_threshold=100):
    weight_wo_fusion = 0.5
    large_val = 1e8
    series_ids = df_0['series_id'].unique()
    out_df = []
    for series_id in series_ids:
        df_0_id = df_0[df_0['series_id']==series_id].copy()
        df_1_id = df_1[df_1['series_id']==series_id].copy()
        df_0_id = df_0_id.sort_values("score", ascending=False).reset_index(drop=True)
        df_1_id = df_1_id.sort_values("score", ascending=False).reset_index(drop=True)
        
        steps_0 = df_0_id['step'].values.copy() # base
        steps_1 = df_1_id['step'].values.copy()
        scores_0 = df_0_id['score'].values.copy() # base
        scores_1 = df_1_id['score'].values.copy()
        not_assigned_df = []
        for step, score in zip(steps_1, scores_1):
            dists = np.abs(steps_0 - step)
            argmin = np.argmin(dists)
            min_dist = dists[argmin]
            if min_dist < distance_threshold:
                f_step = steps_0[argmin]
                f_score = scores_0[argmin]
                add_step = step
                add_score = score
                
                # new_score = (f_score + add_score) / 2
                new_score = (f_score * f_score + add_score * add_score) / (f_score + add_score)
                new_step = (f_step * f_score + add_step * add_score) / (f_score + add_score)
                df_0_id.loc[argmin, "score"] = new_score
                df_0_id.loc[argmin, "step"] = new_step
                steps_0[argmin] = large_val # large val to avoid assign again
            else:
                not_assigned = df_1_id[df_1_id['step']==step].copy()
                not_assigned['score'] = score * weight_wo_fusion # not assigned
                not_assigned_df.append(not_assigned)
        df_0_id.loc[steps_0!=large_val, "score"] *= weight_wo_fusion # not assigned
        out_df.append(df_0_id)
        if len(not_assigned_df) >0:
            not_assigned_df = pd.concat(not_assigned_df)
            out_df.append(not_assigned_df)
    out_df = pd.concat(out_df).reset_index(drop=True) # .reset_index() # .rename(columns={"index": "row_id"})
    return out_df

def round_step(df):
    df["step"] = df["step"].astype(int) + (df["step"] % 6 < 1).astype(int)
    return df



def run_3rd_stage(df_second, pretrained_dir, 
                  mins_offset = [2,4,8,16], #32],#[2, 3, 9, 30, 60, 1440],
                 ):
    def drop_duplicate_1minstep(df):
        df["step_1min"] = df["step"] // cfg.step_for_1min
        df = df.drop_duplicates(["series_id", "event", "step_1min"])
        df = df.drop(columns=["step_1min"])
        return df
    
    def add_offset_events(pred_df):
        """
        Use as many prediction as possible for Average Precision.
        """
        pred_df["offset_from_original"] = 0
        pred_concat = [pred_df.copy()]
        for offset in mins_offset:
            pred_df_c = pred_df.copy()
            pred_df_c["step"] = pred_df_c["step"] - cfg.step_for_1min * offset
            pred_df_c["offset_from_original"] = - offset
            pred_concat.append(pred_df_c)
            pred_df_c = pred_df.copy()
            pred_df_c["step"] = pred_df_c["step"] + cfg.step_for_1min * offset
            pred_df_c["offset_from_original"] = offset
            pred_concat.append(pred_df_c)
        pred_concat = pd.concat(pred_concat, ignore_index=True)
        # If some predictions are in the same minutes, keep the higher.
        pred_concat["step_1min"] = pred_concat["step"] // cfg.step_for_1min
        # pred_concat = pred_concat.sort_values(["series_id", "event", "step_1min", "score"], ascending=False)
        pred_concat = pred_concat.drop_duplicates(["series_id", "event", "step_1min"])
        pred_concat = pred_concat.drop(columns=["step_1min"])
        # pred_concat = pred_concat.drop(columns=["step_1min", "row_id"])
        # pred_concat = pred_concat.reset_index().rename(columns={"index": "row_id"})
        return pred_concat
    
    def add_large_offset_events(pred_df):
        mins_offset = [-1440,-60,60,1440]
        pred_df["offset_from_original"] = 0
        pred_concat = [pred_df.copy()]
        for offset in mins_offset:
            pred_df_c = pred_df.copy()
            pred_df_c["step"] = pred_df_c["step"] + cfg.step_for_1min * offset
            pred_df_c["score"] *= 0.0001 # low score
            pred_df_c["offset_from_original"] = offset
            pred_concat.append(pred_df_c)
        pred_concat = pd.concat(pred_concat, ignore_index=True)
        # If some predictions are in the same minutes, keep the higher.
        pred_concat["step_1min"] = pred_concat["step"] // cfg.step_for_1min
        pred_concat = pred_concat.drop_duplicates(["series_id", "event", "step_1min"])
        pred_concat = pred_concat.drop(columns=["step_1min"])
        return pred_concat
    

    def add_feat_third_stage_ensemble(df, list_pred_files):
        
        num_ensemble = len(list_pred_files)

        out_df = []
        for files in zip(*list_pred_files):
            df_pred_id = pd.read_parquet(files[0])
            cols = [c for c in df_pred_id.columns if "pred_switch" in c] + ["pred_awake"]
            for pf in files[1:]:
                df_pred_id_1 = pd.read_parquet(pf)    
                for c in cols:
                    df_pred_id[c] += df_pred_id_1[c]
            for c in cols:
                df_pred_id[c] /= num_ensemble
            cols = df_pred_id.columns
            new_cols = [col + "_third" for col in cols]
            series_id = files[0].split("id_")[-1].split("_")[0]
            df_series = df[df["series_id"] == series_id].copy()
            steps = df_series["step"].values.astype(int)
            steps = np.clip(steps, 0, len(df_pred_id)-1)
            df_series[new_cols] = df_pred_id.values[steps]
            out_df.append(df_series)
        out_df = pd.concat(out_df, ignore_index=True)
        return out_df      

    
    df_third = add_offset_events(df_second)
    df_third = df_third[df_third["offset_from_original"] != 0]
    
    pred_files_1st_stage = [glob.glob(os.path.join(cfg.pred_dirs[i], "*.parquet")) for i in range(len(cfg.pred_dirs))]
    df_third = add_feat_third_stage_ensemble(df_third, pred_files_1st_stage)
    
    serializer = LGBMSerializer.from_file(pretrained_dir)
    cvbooster = serializer.booster
    encoder = serializer.encoders
    feature_cols = cvbooster.feature_name()[0]
    predicted = np.array(cvbooster.predict(encoder.transform(df_third[feature_cols]))).mean(axis=0)
    # df_third["score"] = df_third["pred_switch10_third"] * 0.085 + predicted * 0.110
    print("changed weights of 3rd stage")
    df_third["score"] = predicted * 0.40
    df_third = df_third[df_third["score"]>0.0008]
    df_third = pd.concat([add_large_offset_events(df_second), 
                          df_third], ignore_index=True)[["series_id", "step", "event", "score"]]
    df_third = df_third.drop_duplicates(["series_id", "event", "step"], keep="first")
    df_third = df_third.sort_values(["series_id", "step"])
    df_third = df_third.reset_index(drop=True).reset_index().rename(columns={"index": "row_id"})
    return df_third

def remove_out_of_range(df, min_step=0):
    return df[df["step"] > min_step].drop(columns=["row_id"]).reset_index(drop=True).reset_index().rename(columns={"index": "row_id"})



In [ ]:
import numpy as np
import pandas as pd
import pandas.api.types
from typing import Dict, List, Tuple

# tolerances in steps
tolerances = {
    "onset":  [12, 36, 60, 90, 120, 150, 180, 240, 300, 360],
    "wakeup": [12, 36, 60, 90, 120, 150, 180, 240, 300, 360],
}
series_id_column_name = "series_id"
time_column_name = "step"
event_column_name = "event"
score_column_name = "score"
use_scoring_intervals = None

def score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        tolerances: Dict[str, List[float]],
        series_id_column_name: str,
        time_column_name: str,
        event_column_name: str,
        score_column_name: str,
        use_scoring_intervals: bool = False,
) -> float:
    
    # Validate metric parameters
    assert len(tolerances) > 0, "Events must have defined tolerances."
    assert set(tolerances.keys()) == set(solution[event_column_name]).difference({'start', 'end'}),\
        (f"Solution column {event_column_name} must contain the same events "
         "as defined in tolerances.")
    assert pd.api.types.is_numeric_dtype(solution[time_column_name]),\
        f"Solution column {time_column_name} must be of numeric type."

    # Validate submission format
    for column_name in [
        series_id_column_name,
        time_column_name,
        event_column_name,
        score_column_name,
    ]:
        if column_name not in submission.columns:
            raise ParticipantVisibleError(f"Submission must have column '{target_name}'.")

    if not pd.api.types.is_numeric_dtype(submission[time_column_name]):
        raise ParticipantVisibleError(
            f"Submission column '{time_column_name}' must be of numeric type."
        )
    if not pd.api.types.is_numeric_dtype(submission[score_column_name]):
        raise ParticipantVisibleError(
            f"Submission column '{score_column_name}' must be of numeric type."
        )

    # Set these globally to avoid passing around a bunch of arguments
    globals()['series_id_column_name'] = series_id_column_name
    globals()['time_column_name'] = time_column_name
    globals()['event_column_name'] = event_column_name
    globals()['score_column_name'] = score_column_name
    globals()['use_scoring_intervals'] = use_scoring_intervals

    return event_detection_ap(solution, submission, tolerances)

def event_detection_ap(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        tolerances: Dict[str, List[float]],
) -> float:

    # Ensure solution and submission are sorted properly
    solution = solution.sort_values([series_id_column_name, time_column_name])
    submission = submission.sort_values([series_id_column_name, time_column_name])

    # Extract scoring intervals.
    if use_scoring_intervals:
        intervals = (
            solution
            .query("event in ['start', 'end']")
            .assign(interval=lambda x: x.groupby([series_id_column_name, event_column_name]).cumcount())
            .pivot(
                index='interval',
                columns=[series_id_column_name, event_column_name],
                values=time_column_name,
            )
            .stack(series_id_column_name)
            .swaplevel()
            .sort_index()
            .loc[:, ['start', 'end']]
            .apply(lambda x: pd.Interval(*x, closed='both'), axis=1)
        )

    # Extract ground-truth events.
    ground_truths = (
        solution
        .query("event not in ['start', 'end']")
        .reset_index(drop=True)
    )

    # Map each event class to its prevalence (needed for recall calculation)
    class_counts = ground_truths.value_counts(event_column_name).to_dict()

    # Create table for detections with a column indicating a match to a ground-truth event
    detections = submission.assign(matched = False)

    # Remove detections outside of scoring intervals
    if use_scoring_intervals:
        detections_filtered = []
        for (det_group, dets), (int_group, ints) in zip(
            detections.groupby(series_id_column_name), intervals.groupby(series_id_column_name)
        ):
            assert det_group == int_group
            detections_filtered.append(filter_detections(dets, ints))
        detections_filtered = pd.concat(detections_filtered, ignore_index=True)
    else:
        detections_filtered = detections

    # Create table of event-class x tolerance x series_id values
    aggregation_keys = pd.DataFrame(
        [(ev, tol, vid)
         for ev in tolerances.keys()
         for tol in tolerances[ev]
         for vid in ground_truths[series_id_column_name].unique()],
        columns=[event_column_name, 'tolerance', series_id_column_name],
    )

    # Create match evaluation groups: event-class x tolerance x series_id
    detections_grouped = (
        aggregation_keys
        .merge(detections_filtered, on=[event_column_name, series_id_column_name], how='left')
        .groupby([event_column_name, 'tolerance', series_id_column_name])
    )
    ground_truths_grouped = (
        aggregation_keys
        .merge(ground_truths, on=[event_column_name, series_id_column_name], how='left')
        .groupby([event_column_name, 'tolerance', series_id_column_name])
    )
    # Match detections to ground truth events by evaluation group
    detections_matched = []
    for key in aggregation_keys.itertuples(index=False):
        dets = detections_grouped.get_group(key)
        gts = ground_truths_grouped.get_group(key)
        detections_matched.append(
            match_detections(dets['tolerance'].iloc[0], gts, dets)
        )
    detections_matched = pd.concat(detections_matched)

    # Compute AP per event x tolerance group
    event_classes = ground_truths[event_column_name].unique()
    ap_table = (
        detections_matched
        .query("event in @event_classes")
        .groupby([event_column_name, 'tolerance']).apply(
            lambda group: average_precision_score(
                group['matched'].to_numpy(),
                group[score_column_name].to_numpy(),
                class_counts[group[event_column_name].iat[0]],
            )
        )
    )
    # Average over tolerances, then over event classes
    mean_ap = ap_table.groupby(event_column_name).mean().sum() / len(event_classes)

    return mean_ap

def match_detections(
        tolerance: float, ground_truths: pd.DataFrame, detections: pd.DataFrame
) -> pd.DataFrame:
    """Match detections to ground truth events. Arguments are taken from a common event x tolerance x series_id evaluation group."""
    detections_sorted = detections.sort_values(score_column_name, ascending=False).dropna()
    is_matched = np.full_like(detections_sorted[event_column_name], False, dtype=bool)
    gts_matched = set()
    for i, det in enumerate(detections_sorted.itertuples(index=False)):
        best_error = tolerance
        best_gt = None

        for gt in ground_truths.itertuples(index=False):
            error = abs(getattr(det, time_column_name) - getattr(gt, time_column_name))
            if error < best_error and gt not in gts_matched:
                best_gt = gt
                best_error = error

        if best_gt is not None:
            is_matched[i] = True
            gts_matched.add(best_gt)

    detections_sorted['matched'] = is_matched

    return detections_sorted

def precision_recall_curve(
        matches: np.ndarray, scores: np.ndarray, p: int
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    if len(matches) == 0:
        return [1], [0], []

    # Sort matches by decreasing confidence
    idxs = np.argsort(scores, kind='stable')[::-1]
    scores = scores[idxs]
    matches = matches[idxs]

    distinct_value_indices = np.where(np.diff(scores))[0]
    threshold_idxs = np.r_[distinct_value_indices, matches.size - 1]
    thresholds = scores[threshold_idxs]

    # Matches become TPs and non-matches FPs as confidence threshold decreases
    tps = np.cumsum(matches)[threshold_idxs]
    fps = np.cumsum(~matches)[threshold_idxs]

    precision = tps / (tps + fps)
    precision[np.isnan(precision)] = 0
    recall = tps / p  # total number of ground truths might be different than total number of matches

    # Stop when full recall attained and reverse the outputs so recall is non-increasing.
    last_ind = tps.searchsorted(tps[-1])
    sl = slice(last_ind, None, -1)

    # Final precision is 1 and final recall is 0
    return np.r_[precision[sl], 1], np.r_[recall[sl], 0], thresholds[sl]

def average_precision_score(matches: np.ndarray, scores: np.ndarray, p: int) -> float:
    precision, recall, _ = precision_recall_curve(matches, scores, p)
    # Compute step integral
    return -np.sum(np.diff(recall) * np.array(precision)[:-1])



In [ ]:
def add_many_events(pred_df):
    """
    Use as many prediction as possible for Average Precision.
    """
    
    mins_offset = [3, 9, 30, 2, 60, 1440]
    score_rate = [0.01, 0.008, 0.005, 0.002, 0.0001, 0.00001]

    pred_concat = [pred_df.copy()]
    for offset, ratio in zip(mins_offset, score_rate):
        pred_df_c = pred_df.copy()
        pred_df_c["step"] = pred_df_c["step"] - cfg.step_for_1min * offset
        pred_df_c["score"] = pred_df_c["score"] * ratio
        pred_concat.append(pred_df_c)
        pred_df_c = pred_df.copy()
        pred_df_c["step"] = pred_df_c["step"] + cfg.step_for_1min * offset
        pred_df_c["score"] = pred_df_c["score"] * ratio
        pred_concat.append(pred_df_c)
    pred_concat = pd.concat(pred_concat, ignore_index=True)

    print(pred_concat.shape)
    # If some predictions are in the same minutes, keep the higher.
    pred_concat["step_1min"] = pred_concat["step"] // cfg.step_for_1min
    pred_concat = pred_concat.sort_values(["series_id", "event", "step_1min", "score"], ascending=False)
    pred_concat = pred_concat.drop_duplicates(["series_id", "event", "step_1min"])
    pred_concat = pred_concat.drop(columns=["step_1min", "row_id"])
    pred_concat = pred_concat.reset_index().rename(columns={"index": "row_id"})
    print(pred_concat.shape)
    return pred_concat


def submit_sample():
    sample = pd.read_csv(cfg.sample_submission_path)
    sample.to_csv('submission.csv', index=False)


In [ ]:
import shutil
pp_params = {
    'awake_threshold': 0.01118579240197301,
    'sleep_threshold': 0.9741809981418406,
    'awake_sleep_diff_threshold': 0.0036734165080717115,
    'peak_kernel_size': 4.689964359639259,
    'peak_threshold': 0.05135826117540523,
    'event_before_length': 68.38372582053368,
    'event_after_length': 99.69200455879107,
    'prior_conf_dev': 0.29102092096947463,
    "averaging_weight": [0.333, 0.333, 0.333, 0.00, 0, 0]
}

if not cfg.is_rerun and not cfg.is_train:
    print("submit sample")
    sample = pd.read_csv(cfg.sample_submission_path)
    sample.to_csv('submission.csv', index=False)
else:
    print("Generating submission directly from Stage 1")
    
    # Run Stage 1 and preprocess
    show_memory_usage("before data preparation")
    data_preparation(cfg.train_path if cfg.is_train else cfg.test_path,
                     load_path_target=None,
                     save_dir=cfg.preprocess_dir)
    show_memory_usage("aft data preparation")
    
    infer_predict_twin(load_dir=cfg.preprocess_dir, save_dir_0=cfg.pred_dirs[0], save_dir_1=cfg.pred_dirs[1], overlap=cfg.pred_overlap)
    shutil.rmtree(cfg.preprocess_dir)
    show_memory_usage("aft inference")

    # Process Stage 1 outputs directly
    out_df = run_postprocess_and_prepare_2nd(cfg.pred_dirs[0], pp_params, save_path="df_second_model.feather")
    show_memory_usage("aft postprocess")
    gc.collect()

    # Save submission
    sub_columns = ["row_id", "series_id", "step", "event", "score"]
    out_df["step"] = out_df["step"].astype(int)
    out_df[sub_columns].to_csv('submission.csv', index=False)
    print("Submission file generated successfully!")
